# **CLUSTERING**

K-MEANS y Elbow Model

In [1]:
!pip install matplotlib seaborn sklearn

In [1]:
#Importamos librerías
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
%matplotlib inline
from sklearn.cluster import KMeans

# Configuración warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importamos el dataset
df = pd.read_csv("")
df_cluster = 

In [ ]:
df_cluster.shape


In [ ]:
df_cluster.head()

In [ ]:
#Hacemos una rápida visualización para ver como son los datos
print(df_cluster)

In [ ]:
#Mostramos las variables y sus datatypes
df_cluster.columns


In [ ]:
df_cluster.dtypes

Elbow Method para ver el número de clusters

In [ ]:
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df_cluster)
    distortions.append(kmeanModel.inertia_)

Mostramos en pantalla y analizamos el **elbow plot**

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion score')
plt.title('The Elbow Method showing the optimal k')
plt.show()

**Método K-MEANS**

In [ ]:
clustering =  KMeans(n_clusters = 2, max_iter = 300)
clustering.fit(df_cluster)

# **Shiloutte Score**

In [10]:
#Importamos librerías
import numpy as np
import pandas as pd
import csv
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [11]:
#Importamos el dataset
dataframe = pd.read_csv('')
df = dataframe.copy(deep=True)

In [12]:
#Usamos el silhouette score
range_n_clusters = list (range(2,10))
print ("Number of clusters from 2 to 9: \n", range_n_clusters)

Number of clusters from 2 to 9: 
 [2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# Ajustamos el modelo para el rango de resultados y mostramos los resultados

for n_clusters in range_n_clusters:
    clusterer = KMeans (n_clusters=n_clusters).fit(df)
    preds = clusterer.predict(df)
    centers = clusterer.cluster_centers_
    score = silhouette_score(df, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))

# **Agregamos al dataset original** 

In [ ]:
df['Cluster'] = clustering.labels_
df.head()

In [ ]:
df.to_csv('cluster_train.csv', sep=';', index=False)

# **Visualización clusters**

In [ ]:
from sklearn.decomposition import PCA 

pca = PCA (n_components=2)
pca_df_cluster = pca.fit_transform(df_cluster)
pca_df_cluster_2 = pd.Dataframe(data=pca_df_cluster, columns = ['Componente1', 'Componente2'])
pca_columna = pd.concat([pca_df_cluster_2, df[['Cluster']]], axis=1)


In [ ]:
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Componente1', fontsize = 15)
ax.set_ylabel('Componente2', fontsize = 15)
ax.set_title('Componentes', fontsize = 20)

color_theme = np.array(["red", "blue", "black"])
ax.scatter(x = pca_columna.Componente1, y = pca_columna.Componente2,
          c=color_theme[pca_columna.Cluster], s = 50)
plt.show()

Agregamos los clusters al dataset test

In [ ]:
df = pd.read_csv("",sep=';')
test, id_column = df.drop(['customerid'],axis=1), df['customerid']
res = clustering.predict(test)
test['customerid'], test['Cluster'] = id_column, res
test.head()